# Supertrend strategie

## Notebook voor het maken van indicators om supertrends te detecteren en hierop te reageren.

Deze strategie is gemaakt voor freqtrade. Freqtrade heeft het detecteren van supertrends echter niet standaard binnen de TA-Lib module waarvan gebruik gemaakt wordt. Daarom dient deze zelf gemaakt te worden met code. Een andere optie is om gebruik te maken van de Pandas_ta module waar supertrend wel in opgenomen is. 
Pandas_ta dient dan wel als module in de freqtrade strategie meegenomen te worden.

Bron: https://www.youtube.com/watch?v=JqEjhBD55NY

Daarna Pandas-ta gebruikt en deze bron nagebouwd: https://www.youtube.com/watch?v=6PbwVZ6HwRE

## Trading rules

De regels zijn als volgt:

Indien de 'supertrend ultra' bullish is:

* indien short, medium & long supertrend bullish (groen) zijn, dan trade met R:R = 1:1,5
* indien short, medium & long supertrend bearish (rood) zijn, dan trade met R:R = 1:1,2

Wanneer bijvoorbeeld de short (& medium) supertrend kortstondig worden gebroken ivm. een pullback en de trend weer voortgezet wordt (doordat alle supertrends weer bullish/bearish zijn), dan opnieuw een trade conform R:R.
Kortom, elke keer als alle supertrends een indicatie geven voor een trend, dan een trade plaatsen.

Indien de 'supertrend ultra' bearish is:

* indien short, medium & long supertrend bearish (rood) zijn, dan trade met R:R = 1:1,5
* indien short, medium & long supertrend bullish (groen) zijn, dan trade met R:R = 1:1,2

De reden voor R:R 1:5 of 1:2 is dat, wanneer korte termijn trends de ultratrend tegenspreken, de kans op reversal mogelijk groter is en eerder winst genomen moet worden om toch een winnende trade te hebben. Bij handmatige backtests viel het mij namelijk op dat in die gevallen een 1:5 ratio vaker tot slechte trades leidde maar dat met 1:2 er toch nog een klein winstje te behalen viel.  
In het geval van ultra trend reversals leid dat wel weer tot te snel winst nemen (1:5 had hier namelijk wel gekund), alleen deze ultra reversals zijn moeilijker te voorspellen.

* Ultra trend: Days 10, Factor 4 (* atr)
* Long trend : Days 10, Factor 1,8
* Medium trend: Days 10, Factor 1,3
* Short trend: Days 10, Factor 0,8

Grofweg gaat het proces om te bepalen wanneer ingestapt moet worden en met welk risicoprofiel als volgt:

1. Bepaal de Ultratrend. Deze bepaald in grote mate het risicoprofiel van de daadwerkelijke trades.
2. Bepaal wanneer long, medium en short trends allemaal bullish zijn.
3. Bepaal het omslagpunt van deze trendverandering. Met deze informatie weet je nu wanneer je moet instappen en met welk risicoprofiel.
4. Bepaal je stoploss prijs. In mijn geval is dat altijd de prijs van de long trend wanneer ingestapt moet worden. Deze moet in een cel komen die niet meer mag wijzigen totdat er weer een trendverandering is.
5. Bepaal Take-profit op basis van de afstand tussen entryprice en long trend de volgende take-profit punten:
        a. Take-profit high risk (ultratrend = 1) R:R = 1:1,5
        b. Take-profit low risk (ultratrend = -1) R:R = 1:1,2
   Ook deze waarden moeten in een cel komen die niet meer veranderd gedurende de trend. Behalve als deze weer wijzigd. Dit zijn ook weer functies die bepalen wat op welk moment gekozen moet worden.
6. Je hebt nu Het entrysignaal, sl prijs en tp prijzen obv risicoprofiel. Nu dient deze als strategie opgenomen worden in freqtrade.

    Als longtrend, mediumtrend, shorttrend zijn long (1)

        Bepaal ultratrend is bullis of bearish
        
        Calculate Stoploss & Takeprofit obv. bullish/bearish ultratrend

        Enter trade

    Als candleclose > takeprofit: 

        Exit trade met winst :-)

    Als candleclose < stoploss: 

        Exit trade met verlies :-(

![Supertrend Tradingview](./images/supertrend_strat.png)

In [51]:
# Importeer de benodigde bibliotheken
import pandas as pd
import pandas_ta as pta
from pandas import DataFrame
import numpy as np

In [2]:
# Load data
df = pd.read_json("./data/ETH_USDT-1d.json")
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.tail(5)

date     open     high      low    close        volume
994 2021-05-28  2742.23  2762.45  2327.38  2412.10  1.554585e+06
995 2021-05-29  2412.34  2574.27  2204.10  2278.74  1.379427e+06
996 2021-05-30  2278.74  2476.42  2180.53  2385.98  1.151113e+06
997 2021-05-31  2385.82  2720.00  2271.44  2706.15  1.307958e+06
998 2021-06-01  2706.15  2740.00  2523.74  2634.57  1.179577e+06

## Supertrend gebruiken van Pandas_ta

Zie ook https://stackoverflow.com/questions/44935269/supertrend-code-using-pandas-python

In [3]:
df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.2.45b0
Total Indicators: 133
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_doji, cdl_inside, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, decay, decreasing, dema, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, kama, kc, kdj, kst, kurtosis, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, ssf, stdev, stoch, stochrsi, supertrend, swma, t3, tema, thermo, trend_return, trima, trix, true_range, tsi, ttm_trend, ui, uo, variance, vidya, vortex, vp, vwap, vwma, wcp, willr, wma, zlm

## 1. Bepaal de Ultratrend

Deze bepaald of high R:R of low R:R gebruikt moet worden.

In [4]:
# Gebruik pandas_ta om de eerste supertrend (Ultra) van de strategie te berekenen (default waarden zijn: period=7, multiplier=3)
# gebruik maken van deze indicator heeft nog niet tot gevolg dat deze ook aan de dataframe wordt toegevoegd. Dat gebeurd in de cel hierna.
pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=10, multiplier=3).tail(5)

SUPERT_10_3.0  SUPERTd_10_3.0  SUPERTl_10_3.0  SUPERTs_10_3.0
994    3748.359457              -1             NaN     3748.359457
995    3748.359457              -1             NaN     3748.359457
996    3701.993094              -1             NaN     3701.993094
997    3701.993094              -1             NaN     3701.993094
998    3701.993094              -1             NaN     3701.993094

In [5]:
help(pta.supertrend)

Help on function supertrend in module pandas_ta.overlap.supertrend:

supertrend(high, low, close, length=None, multiplier=None, offset=None, **kwargs)
    Supertrend (supertrend)
    
    Supertrend is an overlap indicator. It is used to help identify trend
    direction, setting stop loss, identify support and resistance, and/or
    generate buy & sell signals.
    
    Sources:
        http://www.freebsensetips.com/blog/detail/7/What-is-supertrend-indicator-its-calculation
    
    Calculation:
        Default Inputs:
            length=7, multiplier=3.0
        Default Direction:
            Set to +1 or bullish trend at start
    
        MID = multiplier * ATR
        LOWERBAND = HL2 - MID
        UPPERBAND = HL2 + MID
    
        if UPPERBAND[i] < FINAL_UPPERBAND[i-1] and close[i-1] > FINAL_UPPERBAND[i-1]:
            FINAL_UPPERBAND[i] = UPPERBAND[i]
        else:
            FINAL_UPPERBAND[i] = FINAL_UPPERBAND[i-1])
    
        if LOWERBAND[i] > FINAL_LOWERBAND[i-1] and clos

**Tip: Lees eerst de HELP file van de indicator voordat je iets doet.**

Twee dagen zitten klooien met 'period' als argument voor periode van dagen/uren (op basis van een youtube over deze indicator). In werkelijkheid is het argument 'length' voor het bepalen van de periode!

### Verdere uitwerking ultratrend

In [6]:
# # Hierboven zagen we dat kolom "SUPERTd_7_4.0" (d = direction?) aangeeft of er een bearish trend (-1) is of juist bullish trend (1).
# #
# # Dit wetende, ga ik een viertal supertrend kolommen maken in de dataframe die voor elke supertrend (Ultra, Long, Medium, Short) een signaal geven.
# # Nb. aanpassen van period naar een ander getal heeft een bug. De berekende waarde blijf op 7 steken. Daarom blijf ik de default periode gebruiken.
# df['st_ultra'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], period=7, multiplier=3)['SUPERTd_7_3.0']
# df['st_long'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], period=7, multiplier=1.8)['SUPERTd_7_1.8']
# df['st_medium'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], period=7, multiplier=1.3)['SUPERTd_7_1.3']
# df['st_short'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], period=7, multiplier=0.8)['SUPERTd_7_0.8']
# df[['date','st_ultra','st_long','st_medium','st_short']]

Hierboven begon ik voor alle supertrends alvast kolommen te maken. In de Google Collab sheet(https://colab.research.google.com/drive/1OH-q7H7ga8RariwdLxbco-YR8CAvBssj#scrollTo=Ob0eTVi2nnFa) deed ik dit voor alle supertrends tegelijk. Dit was op een gegeven moment verwarrend. In dit notebook wil ik eerst per supertrend eea wegprogrammeren. Daarna worden ze samengevat in conditites voor instappen en uitstappen.

Ultra trend wordt in deze strategie alleen gebruikt voor het bepalen van het TP punt. 

In [7]:
# SUPERTd (direction?) opnemen in een kolom (supertrend ultra direction)
df['st_ultra_d'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=10, multiplier=3)['SUPERTd_10_3.0']
df

date     open     high      low    close        volume  st_ultra_d
0   2018-09-07   229.73   234.69   213.50   215.22  3.983798e+05           1
1   2018-09-08   215.19   220.20   190.01   196.71  3.188598e+05           1
2   2018-09-09   196.87   208.60   185.02   195.81  4.292550e+05           1
3   2018-09-10   196.13   203.42   187.10   197.15  4.184899e+05           1
4   2018-09-11   197.22   200.20   176.16   185.18  4.767458e+05           1
..         ...      ...      ...      ...      ...           ...         ...
994 2021-05-28  2742.23  2762.45  2327.38  2412.10  1.554585e+06          -1
995 2021-05-29  2412.34  2574.27  2204.10  2278.74  1.379427e+06          -1
996 2021-05-30  2278.74  2476.42  2180.53  2385.98  1.151113e+06          -1
997 2021-05-31  2385.82  2720.00  2271.44  2706.15  1.307958e+06          -1
998 2021-06-01  2706.15  2740.00  2523.74  2634.57  1.179577e+06          -1

[999 rows x 7 columns]

In dit geval wordt daarom alleen de direction gebruikt in een later stadium wanneer takeprofit wordt berekend.

## 2. Long, Medium, Short supertrend bepaling

Nu achterhalen wanneer de daadwerkelijke trades gemaakt moeten worden.

In [8]:
# Voor long, medium en short supertrends wordt dezelfde kolom gebruikt om te bepalen of deze bullish of bearish zijn
df['st_long_d'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=10, multiplier=1.8)['SUPERTd_10_1.8']
df['st_medium_d'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=10, multiplier=1.3)['SUPERTd_10_1.3']
df['st_short_d'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=10, multiplier=0.8)['SUPERTd_10_0.8']

# Totaal wordt dit dan:
df[['date','st_ultra_d','st_long_d','st_medium_d','st_short_d']]

date  st_ultra_d  st_long_d  st_medium_d  st_short_d
0   2018-09-07           1          1            1           1
1   2018-09-08           1          1            1           1
2   2018-09-09           1          1            1           1
3   2018-09-10           1          1            1           1
4   2018-09-11           1          1            1           1
..         ...         ...        ...          ...         ...
994 2021-05-28          -1         -1            1           1
995 2021-05-29          -1         -1            1          -1
996 2021-05-30          -1         -1            1          -1
997 2021-05-31          -1         -1            1           1
998 2021-06-01          -1         -1            1           1

[999 rows x 5 columns]

Indien alles 1 is, dan is er een bullish trend waarop ingestapt kan worden. Echter het daadwerkelijke instappen dient eigenlijk te gebeuren als de volgende condities plaatvinden:

* Short en medium supertrends zijn al eerder bullish en vervolgens wordt ook Long supertrend doorbroken (reversal)
* Long en Medium st dienen als support en vervolgens breekt de prijs weer door de short st (shallow retracement)
* Alleen long st is nog support en prijs breekt daarna vervolgens weer bullish door medium en short st (deep retracement) 

Zo zijn er dus verschillende combinaties te maken met elk verschillende toegepaste taktieken (in de vorm van TP en SL).

## 3. Positie van trade bepalen

Op basis van long, medium en short supertrends kunnen we nu de instappositie bepalen

Met de onderstaande if-elif statements kunnen posities worden bepaald. Later nadenken hoe het daadwerkelijke instapsignaal tot stand moet gaan komen.

In [9]:
def get_position(row):
    if row['st_ultra_d'] == 1 and row['st_long_d'] == 1 and row['st_medium_d'] == 1 and row['st_short_d'] == 1:
        val = 'high_r_long' # R:R 1:5 long position
    elif row['st_ultra_d'] == -1 and row['st_long_d'] == 1 and row['st_medium_d'] == 1 and row['st_short_d'] == 1:
        val = 'low_r_long' # R:R 1:2 long position
    else:
        val = 'neutral'
    return val

df['position'] = df.apply(get_position, axis=1)
df.tail(50)

date     open     high      low    close        volume  st_ultra_d  \
949 2021-04-13  2137.86  2318.00  2135.53  2299.19  7.826086e+05           1   
950 2021-04-14  2299.19  2447.29  2281.33  2432.60  9.729316e+05           1   
951 2021-04-15  2432.59  2543.47  2400.00  2514.04  6.819510e+05           1   
952 2021-04-16  2514.04  2548.29  2300.00  2422.67  9.991638e+05           1   
953 2021-04-17  2422.38  2495.00  2310.00  2317.60  6.242323e+05           1   
954 2021-04-18  2317.65  2340.00  1946.80  2235.64  1.475384e+06           1   
955 2021-04-19  2235.65  2300.00  2080.00  2161.12  8.205923e+05           1   
956 2021-04-20  2161.12  2346.62  2055.00  2330.03  9.922408e+05           1   
957 2021-04-21  2330.26  2469.54  2235.16  2357.06  1.177886e+06           1   
958 2021-04-22  2357.06  2644.47  2303.44  2397.94  2.220753e+06           1   
959 2021-04-23  2397.94  2442.89  2107.00  2367.39  2.532242e+06           1   
960 2021-04-24  2367.39  2368.74  2154.80  2213.35  9.976889e+05           1   
961 2021-04-25  2214.08  2356.64  2168.00  2320.49  8.575174e+05           1   
962 2021-04-26  2320.44  2542.80  2303.64  2532.60  1.076469e+06           1   
963 2021-04-27  2532.06  2680.00  2480.29  2666.13  7.717285e+05           1   
964 2021-04-28  2666.13  2760.00  2559.44  2748.24  9.993832e+05           1   
965 2021-04-29  2748.23  2798.00  2668.00  2757.04  8.638900e+05           1   
966 2021-04-30  2757.04  2798.30  2723.09  2772.42  6.953392e+05           1   
967 2021-05-01  2772.42  2954.28  2743.41  2944.77  6.902601e+05           1   
968 2021-05-02  2944.83  2985.00  2852.26  2949.56  5.719825e+05           1   
969 2021-05-03  2949.33  3454.00  2949.32  3431.04  1.329088e+06           1   
970 2021-05-04  3431.04  3527.00  3160.00  3239.01  1.826039e+06           1   
971 2021-05-05  3239.01  3549.34  3200.23  3524.26  1.174817e+06           1   
972 2021-05-06  3524.33  3605.40  3376.47  3489.73  9.205926e+05           1   
973 2021-05-07  3489.74  3587.00  3353.23  3479.53  7.605446e+05           1   
974 2021-05-08  3479.79  3958.33  3449.27  3910.26  1.130468e+06           1   
975 2021-05-09  3910.26  3983.65  3726.08  3923.95  1.000401e+06           1   
976 2021-05-10  3923.95  4208.00  3650.00  3945.88  1.505898e+06           1   
977 2021-05-11  3946.63  4180.22  3771.64  4173.45  1.095927e+06           1   
978 2021-05-12  4173.46  4372.72  3701.58  3823.21  1.601843e+06           1   
979 2021-05-13  3810.15  4034.77  3500.00  3717.91  2.068708e+06           1   
980 2021-05-14  3717.92  4173.28  3691.35  4075.01  1.153390e+06           1   
981 2021-05-15  4075.02  4134.05  3632.00  3646.68  1.039175e+06           1   
982 2021-05-16  3646.24  3879.00  3343.06  3582.10  1.324066e+06           1   
983 2021-05-17  3581.81  3587.06  3102.00  3282.51  1.904012e+06           1   
984 2021-05-18  3282.25  3565.71  3240.00  3375.07  1.287111e+06           1   
985 2021-05-19  3375.08  3441.73  1888.00  2438.92  4.309836e+06          -1   
986 2021-05-20  2438.92  2990.05  2155.00  2768.62  2.942778e+06          -1   
987 2021-05-21  2767.87  2938.54  2101.00  2430.98  2.727032e+06          -1   
988 2021-05-22  2430.98  2485.00  2145.00  2294.87  1.684318e+06          -1   
989 2021-05-23  2295.45  2381.57  1728.74  2096.39  2.914549e+06          -1   
990 2021-05-24  2096.63  2675.00  2079.94  2648.43  2.186633e+06          -1   
991 2021-05-25  2647.92  2750.00  2378.54  2705.11  2.115350e+06          -1   
992 2021-05-26  2705.10  2910.00  2643.00  2884.94  1.583800e+06          -1   
993 2021-05-27  2884.82  2889.74  2633.68  2742.23  1.099030e+06          -1   
994 2021-05-28  2742.23  2762.45  2327.38  2412.10  1.554585e+06          -1   
995 2021-05-29  2412.34  2574.27  2204.10  2278.74  1.379427e+06          -1   
996 2021-05-30  2278.74  2476.42  2180.53  2385.98  1.151113e+06          -1   
997 2021-05-31  2385.82  2720.00  2271.44  2706.15  1.307958e+06          -1   
998 2021-0

In [10]:
# Andere poging tot if then statements maar bovenste werkt gelukkig...
# Zie https://medium.com/swlh/adding-a-column-to-a-pandas-dataframe-based-on-an-if-else-condition-895cfc666548
# # create list with conditions
# conditions = [
#     (['st_ultra_d'] == 1) & (['st_long_d'] == 1) & (['st_medium_d'] == 1) & (['st_short_d'] == 1),
#     (['st_ultra_d'] == -1) & (['st_long_d'] == 1) & (['st_medium_d'] == 1) & (['st_short_d'] == 1),
#     ]

# values = ['high_r_long','low_r_long']

# df['position'] = np.select[conditions, values]
# df.tail(50)

## 4. Entry signaal detectie

Met deze formule detecteer ik of er een verandering van advies is gekomen (huidige cel is anders dan vorige cel). Indien ja, en deze is van type 'high/low_r_positie', dan kan ingestapt worden.

In [11]:
df['advice_changed'] = df['position'].shift(+1) != df['position']
df[['advice_changed','position','date']].tail(30)

advice_changed     position       date
969           False  high_r_long 2021-05-03
970           False  high_r_long 2021-05-04
971           False  high_r_long 2021-05-05
972           False  high_r_long 2021-05-06
973           False  high_r_long 2021-05-07
974           False  high_r_long 2021-05-08
975           False  high_r_long 2021-05-09
976           False  high_r_long 2021-05-10
977           False  high_r_long 2021-05-11
978           False  high_r_long 2021-05-12
979            True      neutral 2021-05-13
980            True  high_r_long 2021-05-14
981           False  high_r_long 2021-05-15
982            True      neutral 2021-05-16
983           False      neutral 2021-05-17
984           False      neutral 2021-05-18
985           False      neutral 2021-05-19
986           False      neutral 2021-05-20
987           False      neutral 2021-05-21
988           False      neutral 2021-05-22
989           False      neutral 2021-05-23
990           False      neutral 2021-05-24
991           False      neutral 2021-05-25
992           False      neutral 2021-05-26
993           False      neutral 2021-05-27
994           False      neutral 2021-05-28
995           False      neutral 2021-05-29
996           False      neutral 2021-05-30
997           False      neutral 2021-05-31
998           False      neutral 2021-06-01

## 5. Stoploss prijs bepalen

De prijs van de long trend is de stoploss prijs van deze strategie. Dwz. Als de koers boven long trend komt, dan wordt price-atr gebruikt voor SL. Deze prijs zou volgens Stoploss & Takeprofit theorie niet meer mogen veranderen.
Wellicht laat ik SL meegroeien met de long supertrend prijs.

Trade close is echter als "close < long trend".

In [12]:
df['stoploss'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=10, multiplier=1.8)['SUPERT_10_1.8']
df[['date','open','position','advice_changed','stoploss']].tail(50)

date     open     position  advice_changed     stoploss
949 2021-04-13  2137.86  high_r_long           False  2005.362134
950 2021-04-14  2299.19  high_r_long           False  2135.174621
951 2021-04-15  2432.59  high_r_long           False  2239.688559
952 2021-04-16  2514.04  high_r_long           False  2239.688559
953 2021-04-17  2422.38      neutral            True  2239.688559
954 2021-04-18  2317.65      neutral           False  2449.508538
955 2021-04-19  2235.65      neutral           False  2449.508538
956 2021-04-20  2161.12      neutral           False  2449.508538
957 2021-04-21  2330.26      neutral           False  2449.508538
958 2021-04-22  2357.06      neutral           False  2449.508538
959 2021-04-23  2397.94      neutral           False  2449.508538
960 2021-04-24  2367.39      neutral           False  2449.508538
961 2021-04-25  2214.08      neutral           False  2449.508538
962 2021-04-26  2320.44  high_r_long            True  2030.550359
963 2021-04-27  2532.06  high_r_long           False  2190.794523
964 2021-04-28  2666.13  high_r_long           False  2273.203771
965 2021-04-29  2748.23  high_r_long           False  2361.735394
966 2021-04-30  2757.04  high_r_long           False  2413.019054
967 2021-05-01  2772.42  high_r_long           False  2497.980049
968 2021-05-02  2944.83  high_r_long           False  2578.958344
969 2021-05-03  2949.33  high_r_long           False  2805.113110
970 2021-05-04  3431.04  high_r_long           False  2920.547799
971 2021-05-05  3239.01  high_r_long           False  2931.288219
972 2021-05-06  3524.33  high_r_long           False  3050.580497
973 2021-05-07  3489.74  high_r_long           False  3050.580497
974 2021-05-08  3479.79  high_r_long           False  3217.611313
975 2021-05-09  3910.26  high_r_long           False  3370.932581
976 2021-05-10  3923.95  high_r_long           False  3393.020823
977 2021-05-11  3946.63  high_r_long           False  3420.004341
978 2021-05-12  4173.46  high_r_long           False  3420.004341
979 2021-05-13  3810.15      neutral            True  3420.004341
980 2021-05-14  3717.92  high_r_long            True  3420.004341
981 2021-05-15  4075.02  high_r_long           False  3420.004341
982 2021-05-16  3646.24      neutral            True  3420.004341
983 2021-05-17  3581.81      neutral           False  4085.032043
984 2021-05-18  3282.25      neutral           False  4085.032043
985 2021-05-19  3375.08      neutral           False  3597.108075
986 2021-05-20  2438.92      neutral           False  3561.852767
987 2021-05-21  2767.87      neutral           False  3560.922191
988 2021-05-22  2430.98      neutral           False  3313.236971
989 2021-05-23  2295.45      neutral           False  3071.077674
990 2021-05-24  2096.63      neutral           False  3071.077674
991 2021-05-25  2647.92      neutral           False  3071.077674
992 2021-05-26  2705.10      neutral           False  3071.077674
993 2021-05-27  2884.82      neutral           False  3071.077674
994 2021-05-28  2742.23      neutral           False  3071.077674
995 2021-05-29  2412.34      neutral           False  3071.077674
996 2021-05-30  2278.74      neutral           False  3071.077674
997 2021-05-31  2385.82      neutral           False  3071.077674
998 2021-06-01  2706.15      neutral           False  3071.077674

## 6. Take profit bepalen

Take profit high reward is als ultratrend == 1: ((Close vorige candle - stoploss) * 1,5) + current candle open

Take profit low reward is als ultratrend == -1: ((Close vorige candle - stoploss) * 1,2) + current candle open

In [13]:
# def get_position(row):
#     if row['st_ultra_d'] == 1 and row['st_long_d'] == 1 and row['st_medium_d'] == 1 and row['st_short_d'] == 1:
#         val = 'high_r_long' # R:R 1:5 long position
#     elif row['st_ultra_d'] == -1 and row['st_long_d'] == 1 and row['st_medium_d'] == 1 and row['st_short_d'] == 1:
#         val = 'low_r_long' # R:R 1:2 long position
#     else:
#         val = 'neutral'
#     return val

# df['position'] = df.apply(get_position, axis=1)
# df.tail(50)

In [14]:
def take_profit(row):
    if row['position'] == 'high_r_long' and row['advice_changed'] == True:
        val = ((row['close']-row['stoploss']) * 1.5) + row['close']
    if row['position'] == 'low_r_long' and row['advice_changed'] == True:
        val = ((row['close']-row['stoploss']) * 1.2) + row['close']

        print(row[['date','open','close','position','advice_changed','stoploss']], val)

# print(df.apply(take_profit, axis=1))
df['takeprofit'] = df.apply(take_profit, axis=1)
df[['date','open','close','position','advice_changed','stoploss','takeprofit']]

date              2018-12-18 00:00:00
open                            94.03
close                          100.39
position                   low_r_long
advice_changed                   True
stoploss                    80.522162
Name: 102, dtype: object 124.231406155946
date              2019-02-08 00:00:00
open                           104.48
close                          119.25
position                   low_r_long
advice_changed                   True
stoploss                   100.987433
Name: 154, dtype: object 141.16507992031413
date              2019-03-29 00:00:00
open                           137.85
close                          143.25
position                   low_r_long
advice_changed                   True
stoploss                   131.843391
Name: 203, dtype: object 156.93793069454907
date              2019-09-14 00:00:00
open                           180.96
close                          188.13
position                   low_r_long
advice_changed                   T

date     open    close     position  advice_changed     stoploss  \
0   2018-09-07   229.73   215.22  high_r_long            True     0.000000   
1   2018-09-08   215.19   196.71  high_r_long           False          NaN   
2   2018-09-09   196.87   195.81  high_r_long           False          NaN   
3   2018-09-10   196.13   197.15  high_r_long           False          NaN   
4   2018-09-11   197.22   185.18  high_r_long           False          NaN   
..         ...      ...      ...          ...             ...          ...   
994 2021-05-28  2742.23  2412.10      neutral           False  3071.077674   
995 2021-05-29  2412.34  2278.74      neutral           False  3071.077674   
996 2021-05-30  2278.74  2385.98      neutral           False  3071.077674   
997 2021-05-31  2385.82  2706.15      neutral           False  3071.077674   
998 2021-06-01  2706.15  2634.57      neutral           False  3071.077674   

    takeprofit  
0         None  
1         None  
2         None  
3         None  
4         None  
..         ...  
994       None  
995       None  
996       None  
997       None  
998       None  

[999 rows x 7 columns]

Bovenste werkt bijna goed. Hieronder verder voor de defintieve versie

Onderstaande code checked op signalen en veranderd op basis daarvan takeprofit points. Als er een conditie veranderd, veranderd ook de takeprofit. Als er geen signaal is, dan veranderd de takeprofit niet en neemt deze de waarde van de voorgaande cel over.

In [48]:
for i, row in df.iterrows():
    if row['position'] == 'high_r_long' and row['advice_changed'] == True:
        df.loc[i,'takeprofit'] = ((row['close']-row['stoploss']) * 1.5) + row['close']
    elif row['position'] == 'low_r_long' and row['advice_changed'] == True:
        df.loc[i,'takeprofit'] = ((row['close']-row['stoploss']) * 1.2) + row['close']
    else:
        df.loc[i,'takeprofit'] = df.loc[i-1,'takeprofit']
    
df[['date','open','close','position','advice_changed','stoploss','takeprofit']][df['advice_changed']==True].tail(25)

date     open    close     position  advice_changed     stoploss  \
802 2020-11-17   460.51   482.15  high_r_long            True   429.690674   
810 2020-11-25   604.98   569.50      neutral            True   539.589728   
815 2020-11-30   576.32   616.66  high_r_long            True   515.493535   
819 2020-12-04   615.84   567.24      neutral            True   525.135127   
831 2020-12-16   588.78   637.18  high_r_long            True   545.688599   
836 2020-12-21   638.10   607.58      neutral            True   593.062332   
842 2020-12-27   637.44   685.11  high_r_long            True   585.649552   
857 2021-01-11  1254.24  1087.01      neutral            True  1034.170205   
864 2021-01-18  1232.50  1258.45  high_r_long            True  1034.170205   
867 2021-01-21  1376.72  1110.96      neutral            True  1084.368782   
870 2021-01-24  1233.46  1392.51  high_r_long            True  1084.368782   
873 2021-01-27  1366.54  1239.03      neutral            True  1088.512519   
879 2021-02-02  1374.09  1511.87  high_r_long            True  1189.939258   
899 2021-02-22  1933.54  1777.58      neutral            True  1704.716355   
912 2021-03-07  1650.68  1726.16   low_r_long            True  1400.101273   
918 2021-03-13  1766.13  1922.35  high_r_long            True  1566.170586   
927 2021-03-22  1783.23  1680.97      neutral            True  1624.203028   
934 2021-03-29  1687.16  1816.74  high_r_long            True  1552.624964   
943 2021-04-07  2112.22  1963.47      neutral            True  1875.901972   
946 2021-04-10  2067.19  2133.49  high_r_long            True  1903.217353   
953 2021-04-17  2422.38  2317.60      neutral            True  2239.688559   
962 2021-04-26  2320.44  2532.60  high_r_long            True  2030.550359   
979 2021-05-13  3810.15  3717.91      neutral            True  3420.004341   
980 2021-05-14  3717.92  4075.01  high_r_long            True  3420.004341   
982 2021-05-16  3646.24  3582.10      neutral            True  3420.004341   

      takeprofit  
802   560.838989  
810   560.838989  
815   768.409697  
819   768.409697  
831   774.417101  
836   774.417101  
842   834.300672  
857   834.300672  
864  1594.869693  
867  1594.869693  
870  1854.721826  
873  1854.721826  
879  1994.766113  
899  1994.766113  
912  2117.430473  
918  2456.619121  
927  2456.619121  
934  2212.912554  
943  2212.912554  
946  2478.898970  
953  2478.898970  
962  3285.674461  
979  3285.674461  
980  5057.518489  
982  5057.518489

In [49]:
for i, row in df.iterrows():
    if row['position'] == 'high_r_long' and row['advice_changed'] == True:
        df.loc[i,'takeprofit'] = ((row['close']-row['stoploss']) * 1.5) + row['close']
    elif row['position'] == 'low_r_long' and row['advice_changed'] == True:
        df.loc[i,'takeprofit'] = ((row['close']-row['stoploss']) * 1.2) + row['close']
    else:
        df.loc[i,'takeprofit'] = df.loc[i-1,'takeprofit']

df[['date','open','close','position','advice_changed','stoploss','takeprofit']].tail(50)#[df['advice_changed']==True].tail(25)

date     open    close     position  advice_changed     stoploss  \
949 2021-04-13  2137.86  2299.19  high_r_long           False  2005.362134   
950 2021-04-14  2299.19  2432.60  high_r_long           False  2135.174621   
951 2021-04-15  2432.59  2514.04  high_r_long           False  2239.688559   
952 2021-04-16  2514.04  2422.67  high_r_long           False  2239.688559   
953 2021-04-17  2422.38  2317.60      neutral            True  2239.688559   
954 2021-04-18  2317.65  2235.64      neutral           False  2449.508538   
955 2021-04-19  2235.65  2161.12      neutral           False  2449.508538   
956 2021-04-20  2161.12  2330.03      neutral           False  2449.508538   
957 2021-04-21  2330.26  2357.06      neutral           False  2449.508538   
958 2021-04-22  2357.06  2397.94      neutral           False  2449.508538   
959 2021-04-23  2397.94  2367.39      neutral           False  2449.508538   
960 2021-04-24  2367.39  2213.35      neutral           False  2449.508538   
961 2021-04-25  2214.08  2320.49      neutral           False  2449.508538   
962 2021-04-26  2320.44  2532.60  high_r_long            True  2030.550359   
963 2021-04-27  2532.06  2666.13  high_r_long           False  2190.794523   
964 2021-04-28  2666.13  2748.24  high_r_long           False  2273.203771   
965 2021-04-29  2748.23  2757.04  high_r_long           False  2361.735394   
966 2021-04-30  2757.04  2772.42  high_r_long           False  2413.019054   
967 2021-05-01  2772.42  2944.77  high_r_long           False  2497.980049   
968 2021-05-02  2944.83  2949.56  high_r_long           False  2578.958344   
969 2021-05-03  2949.33  3431.04  high_r_long           False  2805.113110   
970 2021-05-04  3431.04  3239.01  high_r_long           False  2920.547799   
971 2021-05-05  3239.01  3524.26  high_r_long           False  2931.288219   
972 2021-05-06  3524.33  3489.73  high_r_long           False  3050.580497   
973 2021-05-07  3489.74  3479.53  high_r_long           False  3050.580497   
974 2021-05-08  3479.79  3910.26  high_r_long           False  3217.611313   
975 2021-05-09  3910.26  3923.95  high_r_long           False  3370.932581   
976 2021-05-10  3923.95  3945.88  high_r_long           False  3393.020823   
977 2021-05-11  3946.63  4173.45  high_r_long           False  3420.004341   
978 2021-05-12  4173.46  3823.21  high_r_long           False  3420.004341   
979 2021-05-13  3810.15  3717.91      neutral            True  3420.004341   
980 2021-05-14  3717.92  4075.01  high_r_long            True  3420.004341   
981 2021-05-15  4075.02  3646.68  high_r_long           False  3420.004341   
982 2021-05-16  3646.24  3582.10      neutral            True  3420.004341   
983 2021-05-17  3581.81  3282.51      neutral           False  4085.032043   
984 2021-05-18  3282.25  3375.07      neutral           False  4085.032043   
985 2021-05-19  3375.08  2438.92      neutral           False  3597.108075   
986 2021-05-20  2438.92  2768.62      neutral           False  3561.852767   
987 2021-05-21  2767.87  2430.98      neutral           False  3560.922191   
988 2021-05-22  2430.98  2294.87      neutral           False  3313.236971   
989 2021-05-23  2295.45  2096.39      neutral           False  3071.077674   
990 2021-05-24  2096.63  2648.43      neutral           False  3071.077674   
991 2021-05-25  2647.92  2705.11      neutral           False  3071.077674   
992 2021-05-26  2705.10  2884.94      neutral           False  3071.077674   
993 2021-05-27  2884.82  2742.23      neutral           False  3071.077674   
994 2021-05-28  2742.23  2412.10      neutral           False  3071.077674   
995 2021-05-29  2412.34  2278.74      neutral           False  3071.077674   
996 2021-05-30  2278.74  2385.98      neutral           False  3071.077674   
997 2021-05-31  2385.82  2706.15      neutral           False  3071.077674   
998 2021-06-01  2706.15  2634.57      neutral           False  3071.077674   

      takeprofit  
949  2478.89

Bovenstaande werkende code! Is afkomstig van:https://stackoverflow.com/questions/44935269/supertrend-code-using-pandas-python

```
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import pandas as pd

data =yf.download("ACC.NS", period="1d",interval="5m")
data=data.reset_index(drop=True)

data['tr0'] = abs(data["High"] - data["Low"])
data['tr1'] = abs(data["High"] - data["Close"].shift(1))
data['tr2'] = abs(data["Low"]- data["Close"].shift(1))
data["TR"] = round(data[['tr0', 'tr1', 'tr2']].max(axis=1),2)
data["ATR"]=0.00
data['BUB']=0.00
data["BLB"]=0.00
data["FUB"]=0.00
data["FLB"]=0.00
data["ST"]=0.00

# Calculating ATR 
for i, row in data.iterrows():
    if i == 0:
        data.loc[i,'ATR'] = 0.00#data['ATR'].iat[0]
    else:
        data.loc[i,'ATR'] = ((data.loc[i-1,'ATR'] * 13)+data.loc[i,'TR'])/14

data['BUB'] = round(((data["High"] + data["Low"]) / 2) + (2 * data["ATR"]),2)
data['BLB'] = round(((data["High"] + data["Low"]) / 2) - (2 * data["ATR"]),2)


# FINAL UPPERBAND = IF( (Current BASICUPPERBAND < Previous FINAL UPPERBAND) or (Previous Close > Previous FINAL UPPERBAND))
#                     THEN (Current BASIC UPPERBAND) ELSE Previous FINALUPPERBAND)

```

## 7. Exit bepalen.

Als candle df['close'] > df['takeprofit']: exit met winst :-)

Als candle df['close'] < df['stoploss']: exit met verlies :-(


In [55]:
df['supertrend'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], length=10, multiplier=1.8)['SUPERT_10_1.8']
df

date     open     high      low    close        volume  st_ultra_d  \
0   2018-09-07   229.73   234.69   213.50   215.22  3.983798e+05           1   
1   2018-09-08   215.19   220.20   190.01   196.71  3.188598e+05           1   
2   2018-09-09   196.87   208.60   185.02   195.81  4.292550e+05           1   
3   2018-09-10   196.13   203.42   187.10   197.15  4.184899e+05           1   
4   2018-09-11   197.22   200.20   176.16   185.18  4.767458e+05           1   
..         ...      ...      ...      ...      ...           ...         ...   
994 2021-05-28  2742.23  2762.45  2327.38  2412.10  1.554585e+06          -1   
995 2021-05-29  2412.34  2574.27  2204.10  2278.74  1.379427e+06          -1   
996 2021-05-30  2278.74  2476.42  2180.53  2385.98  1.151113e+06          -1   
997 2021-05-31  2385.82  2720.00  2271.44  2706.15  1.307958e+06          -1   
998 2021-06-01  2706.15  2740.00  2523.74  2634.57  1.179577e+06          -1   

     st_long_d  st_medium_d  st_short_d     position  advice_changed  \
0            1            1           1  high_r_long            True   
1            1            1           1  high_r_long           False   
2            1            1           1  high_r_long           False   
3            1            1           1  high_r_long           False   
4            1            1           1  high_r_long           False   
..         ...          ...         ...          ...             ...   
994         -1            1           1      neutral           False   
995         -1            1          -1      neutral           False   
996         -1            1          -1      neutral           False   
997         -1            1           1      neutral           False   
998         -1            1           1      neutral           False   

        stoploss   takeprofit   supertrend  
0       0.000000   538.050000     0.000000  
1            NaN   538.050000          NaN  
2            NaN   538.050000          NaN  
3            NaN   538.050000          NaN  
4            NaN   538.050000          NaN  
..           ...          ...          ...  
994  3071.077674  5057.518489  3071.077674  
995  3071.077674  5057.518489  3071.077674  
996  3071.077674  5057.518489  3071.077674  
997  3071.077674  5057.518489  3071.077674  
998  3071.077674  5057.518489  3071.077674  

[999 rows x 15 columns]

# Volledige code

Hieronder de volledige freqtrade code van de strategie die boven is uitgewerkt:

```
# --- Do not remove these libs ---
from freqtrade.strategy.interface import IStrategy
from pandas import DataFrame
# --------------------------------
import talib.abstract as ta
import pandas_ta as pta
import freqtrade.vendor.qtpylib.indicators as qtpylib
# ---------- Commands -----------
# Commands for backtesting etc.:
# /opt/freqtrade/.env/bin/freqtrade backtesting --config user_data/configs/backtest_conf.json --strategy new_strat
# /opt/freqtrade/.env/bin/freqtrade backtesting --config user_data/configs/backtest_conf.json --timerange=20210201-20210310 --strategy new_strat --export trades --export-filename=user_data/backtest_results/simple_strat_30m_test.json
# /opt/freqtrade/.env/bin/freqtrade plot-dataframe --config user_data/configs/backtest_conf.json --strategy new_strat  --export-filename=user_data/backtest_results/new_strat-2021-05-13_19-29-25.json
# /opt/freqtrade/.env/bin/freqtrade plot-profit --config user_data/configs/backtest_conf.json  --strategy new_strat --export-filename=user_data/backtest_results/new_strat-2021-05-13_19-29-25.json
# --------------------------------

class my_supertrend_strat(IStrategy):
    stoploss = -0.25
    timeframe = "1d"

    order_types = {
        "buy": "limit",
        "sell": "limit",
        "emergencysell": "market",
        "stoploss": "market",
        "stoploss_on_exchange": False,
        "stoploss_on_exchange_interval": 60,
        "stoploss_on_exchange_limit_ratio": 0.99,
    }

    def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        # Indicatoren
        dataframe['st_ultra_d'] = pta.supertrend(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'], length=10, multiplier=3)['SUPERTd_10_3.0']
        dataframe['st_long_d'] = pta.supertrend(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'], length=10, multiplier=1.8)['SUPERTd_10_1.8']
        dataframe['st_medium_d'] = pta.supertrend(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'], length=10, multiplier=1.3)['SUPERTd_10_1.3']
        dataframe['st_short_d'] = pta.supertrend(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'], length=10, multiplier=0.8)['SUPERTd_10_0.8']

        # Trade R:R bepalen
        def get_position(row):
            if row['st_ultra_d'] == 1 and row['st_long_d'] == 1 and row['st_medium_d'] == 1 and row['st_short_d'] == 1:
                val = 'high_r_long' # R:R 1:5 long position
            elif row['st_ultra_d'] == -1 and row['st_long_d'] == 1 and row['st_medium_d'] == 1 and row['st_short_d'] == 1:
                val = 'low_r_long' # R:R 1:2 long position
            else:
                val = 'neutral'
            return val

        dataframe['position'] = dataframe.apply(get_position, axis=1)

        # Verandering advies detectie
        dataframe['advice_changed'] = dataframe['position'].shift(+1) != dataframe['position']

        # Stoploss bepalen
        dataframe['stoploss'] = pta.supertrend(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'], length=10, multiplier=1.8)['SUPERT_10_1.8']
        
        # Takeprofit
        for i, row in dataframe.iterrows():
            if row['position'] == 'high_r_long' and row['advice_changed'] == True:
                dataframe.loc[i,'takeprofit'] = ((row['close']-row['stoploss']) * 1.5) + row['close']
            elif row['position'] == 'low_r_long' and row['advice_changed'] == True:
                dataframe.loc[i,'takeprofit'] = ((row['close']-row['stoploss']) * 1.2) + row['close']
            else:
                dataframe.loc[i,'takeprofit'] = dataframe.loc[i-1,'takeprofit']
        
        # Print stuff
        # print(dataframe.tail(25))
        return dataframe

    def populate_buy_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
            ((dataframe["position"] == "high_r_long") | (dataframe["position"] == "high_r_long"))
            & (dataframe["advice_changed"] == True)
            ),
            "buy",
        ] = 1

        return dataframe

    def populate_sell_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
            (dataframe['close'] > dataframe['takeprofit'])
            | (dataframe['close'] < dataframe['stoploss'])
            ),
            "sell",
        ] = 1
        return dataframe
```

# Conclusie

Het idee was goed maar traden met deze strategie levert helaas minder op dan ik had gehoopt. Uiteindelijk heb ik meer losses dan wins en aangezien de losses zoveel profit opsnoepen, is het een vrij risicovolle strategie. Mogelijk valt er nog iets te tweaken.

***1 hour timeframe***
```
Result for strategy my_supertrend_strat
========================================================== BACKTESTING REPORT =========================================================
|     Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|----------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
| ETH/USDT |    429 |           0.17 |          71.64 |           143.419 |          14.34 |       17:13:00 |   173     0   256  40.3 |
|    TOTAL |    429 |           0.17 |          71.64 |           143.419 |          14.34 |       17:13:00 |   173     0   256  40.3 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |     429 |    173     0   256  40.3 |           0.17 |          71.64 |           143.419 |          71.64 |
====================================================== LEFT OPEN TRADES REPORT ======================================================
|   Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|--------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|  TOTAL |      0 |           0.00 |           0.00 |             0.000 |           0.00 |           0:00 |     0     0     0     0 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-06-02 11:00:00 |
| Max open trades       | 1                   |
|                       |                     |
| Total trades          | 429                 |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 1143.419 USDT       |
| Absolute profit       | 143.419 USDT        |
| Total profit %        | 14.34%              |
| Trades per day        | 0.43                |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 85800.000 USDT      |
|                       |                     |
| Best Pair             | ETH/USDT 71.64%     |
| Worst Pair            | ETH/USDT 71.64%     |
| Best trade            | ETH/USDT 18.16%     |
| Worst trade           | ETH/USDT -12.45%    |
| Best day              | 36.356 USDT         |
| Worst day             | -24.926 USDT        |
| Days win/draw/lose    | 162 / 603 / 234     |
| Avg. Duration Winners | 20:01:00            |
| Avg. Duration Loser   | 15:20:00            |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 0                   |
|                       |                     |
| Min balance           | 877.552 USDT        |
| Max balance           | 1178.122 USDT       |
| Drawdown              | 63.99%              |
| Drawdown              | 128.108 USDT        |
| Drawdown high         | 5.660 USDT          |
| Drawdown low          | -122.448 USDT       |
| Drawdown Start        | 2018-09-13 14:00:00 |
| Drawdown End          | 2020-01-25 01:00:00 |
| Market change         | 1073.4%             |
===============================================

```

***1 day timeframe***
```
Result for strategy my_supertrend_strat
=========================================================== BACKTESTING REPORT ==========================================================
|     Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |
|----------+--------+----------------+----------------+-------------------+----------------+------------------+-------------------------|
| ETH/USDT |     27 |           4.50 |         121.48 |           243.205 |          24.32 | 14 days, 9:47:00 |    13     0    14  48.1 |
|    TOTAL |     27 |           4.50 |         121.48 |           243.205 |          24.32 | 14 days, 9:47:00 |    13     0    14  48.1 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |      26 |     13     0    13  50.0 |           5.64 |         146.63 |           293.555 |         146.63 |
|     stop_loss |       1 |      0     0     1     0 |         -25.15 |         -25.15 |           -50.35  |         -25.15 |
====================================================== LEFT OPEN TRADES REPORT ======================================================
|   Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|--------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|  TOTAL |      0 |           0.00 |           0.00 |             0.000 |           0.00 |           0:00 |     0     0     0     0 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-05-31 00:00:00 |
| Max open trades       | 1                   |
|                       |                     |
| Total trades          | 27                  |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 1243.205 USDT       |
| Absolute profit       | 243.205 USDT        |
| Total profit %        | 24.32%              |
| Trades per day        | 0.03                |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 5400.000 USDT       |
|                       |                     |
| Best Pair             | ETH/USDT 121.48%    |
| Worst Pair            | ETH/USDT 121.48%    |
| Best trade            | ETH/USDT 42.52%     |
| Worst trade           | ETH/USDT -25.15%    |
| Best day              | 85.117 USDT         |
| Worst day             | -50.350 USDT        |
| Days win/draw/lose    | 13 / 923 / 14       |
| Avg. Duration Winners | 14 days, 22:09:00   |
| Avg. Duration Loser   | 13 days, 22:17:00   |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 0                   |
|                       |                     |
| Min balance           | 917.702 USDT        |
| Max balance           | 1282.474 USDT       |
| Drawdown              | 40.77%              |
| Drawdown              | 81.625 USDT         |
| Drawdown high         | 15.885 USDT         |
| Drawdown low          | -65.740 USDT        |
| Drawdown Start        | 2019-06-26 00:00:00 |
| Drawdown End          | 2019-11-22 00:00:00 |
| Market change         | 1157.39%            |
===============================================
```

***1 day timeframe en meerdere coins***
```
Result for strategy my_supertrend_strat
============================================================ BACKTESTING REPORT ===========================================================
|      Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|-----------+--------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|  LTC/USDT |      6 |          29.77 |         178.59 |           357.538 |          35.75 | 20 days, 12:00:00 |     4     0     2  66.7 |
|  TRX/USDT |      5 |          35.70 |         178.50 |           357.354 |          35.74 |  14 days, 4:48:00 |     5     0     0   100 |
|  ADA/USDT |     14 |          11.49 |         160.82 |           321.969 |          32.20 | 17 days, 17:09:00 |     7     0     7  50.0 |
| LINK/USDT |      9 |          16.26 |         146.37 |           293.036 |          29.30 | 18 days, 13:20:00 |     6     0     3  66.7 |
|  VET/USDT |      5 |          26.43 |         132.17 |           264.613 |          26.46 | 19 days, 19:12:00 |     4     0     1  80.0 |
|  ETH/USDT |     19 |           6.64 |         126.10 |           252.451 |          25.25 | 14 days, 20:13:00 |    10     0     9  52.6 |
| ATOM/USDT |     10 |          11.72 |         117.19 |           234.615 |          23.46 |  15 days, 9:36:00 |     4     0     6  40.0 |
|  XRP/USDT |      4 |          16.92 |          67.66 |           135.464 |          13.55 |  14 days, 6:00:00 |     2     0     2  50.0 |
| DOGE/USDT |      6 |           5.52 |          33.11 |            66.295 |           6.63 |  5 days, 20:00:00 |     3     0     3  50.0 |
|  BNB/USDT |     16 |           1.79 |          28.59 |            57.230 |           5.72 | 17 days, 22:30:00 |     5     0    11  31.2 |
|  EOS/USDT |      2 |          12.31 |          24.62 |            49.285 |           4.93 |  15 days, 0:00:00 |     1     0     1  50.0 |
| AAVE/USDT |      2 |          10.12 |          20.24 |            40.524 |           4.05 |   7 days, 0:00:00 |     1     0     1  50.0 |
|  BTC/USDT |     11 |           1.31 |          14.37 |            28.770 |           2.88 |  15 days, 8:44:00 |     4     0     7  36.4 |
|  BCH/USDT |      5 |          -0.69 |          -3.44 |            -6.893 |          -0.69 | 10 days, 14:24:00 |     1     0     4  20.0 |
|  UNI/USDT |      4 |          -7.55 |         -30.22 |           -60.499 |          -6.05 | 14 days, 18:00:00 |     1     0     3  25.0 |
|  YFI/USDT |      2 |         -21.44 |         -42.87 |           -85.829 |          -8.58 |   6 days, 0:00:00 |     0     0     2     0 |
|     TOTAL |    120 |           9.60 |        1151.81 |          2305.922 |         230.59 | 15 days, 12:00:00 |    58     0    62  48.3 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |     110 |     58     0    52  52.7 |          12.76 |        1403.31 |           2809.42 |         467.77 |
|     stop_loss |      10 |      0     0    10     0 |         -25.15 |        -251.5  |           -503.5  |         -83.83 |
====================================================== LEFT OPEN TRADES REPORT ======================================================
|   Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|--------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|  TOTAL |      0 |           0.00 |           0.00 |             0.000 |           0.00 |           0:00 |     0     0     0     0 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-05-31 00:00:00 |
| Max open trades       | 3                   |
|                       |                     |
| Total trades          | 120                 |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 3305.922 USDT       |
| Absolute profit       | 2305.922 USDT       |
| Total profit %        | 230.59%             |
| Trades per day        | 0.12                |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 24000.000 USDT      |
|                       |                     |
| Best Pair             | LTC/USDT 178.59%    |
| Worst Pair            | YFI/USDT -42.87%    |
| Best trade            | ADA/USDT 102.64%    |
| Worst trade           | LINK/USDT -25.15%   |
| Best day              | 205.487 USDT        |
| Worst day             | -72.162 USDT        |
| Days win/draw/lose    | 55 / 847 / 48       |
| Avg. Duration Winners | 17 days, 2:54:00    |
| Avg. Duration Loser   | 13 days, 23:37:00   |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 4591                |
|                       |                     |
| Min balance           | 738.368 USDT        |
| Max balance           | 3390.667 USDT       |
| Drawdown              | 128.26%             |
| Drawdown              | 256.770 USDT        |
| Drawdown high         | -4.862 USDT         |
| Drawdown low          | -261.632 USDT       |
| Drawdown Start        | 2018-10-12 00:00:00 |
| Drawdown End          | 2019-02-05 00:00:00 |
| Market change         | 1627.12%            |
===============================================
```

Winrate is op zich goed als van meerdere coins gebruik gemaakt wordt. Echter de winsten zijn niet spectaculair... Met een R:R van 1:2 en 1:1.5 is het resultaat.

```
Result for strategy my_supertrend_strat
============================================================ BACKTESTING REPORT ===========================================================
|      Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|-----------+--------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|  ADA/USDT |     15 |          14.69 |         220.30 |           441.031 |          44.10 | 18 days, 14:24:00 |     8     0     7  53.3 |
|  TRX/USDT |      3 |          46.98 |         140.93 |           282.132 |          28.21 | 20 days, 16:00:00 |     3     0     0   100 |
|  LTC/USDT |      4 |          33.34 |         133.35 |           266.967 |          26.70 | 26 days, 12:00:00 |     3     0     1  75.0 |
|  ETH/USDT |     17 |           7.38 |         125.52 |           251.294 |          25.13 |  17 days, 2:49:00 |     8     0     9  47.1 |
|  VET/USDT |      6 |          19.72 |         118.29 |           236.824 |          23.68 | 12 days, 20:00:00 |     3     0     3  50.0 |
|  BCH/USDT |      3 |          38.86 |         116.59 |           233.408 |          23.34 |  13 days, 8:00:00 |     2     0     1  66.7 |
| LINK/USDT |     10 |           8.81 |          88.09 |           176.349 |          17.63 |  18 days, 2:24:00 |     6     0     4  60.0 |
|  BNB/USDT |     14 |           4.34 |          60.75 |           121.614 |          12.16 | 18 days, 22:17:00 |     5     0     9  35.7 |
| AAVE/USDT |      2 |          18.79 |          37.57 |            75.223 |           7.52 |  9 days, 12:00:00 |     1     0     1  50.0 |
|  BTC/USDT |     12 |           2.56 |          30.72 |            61.505 |           6.15 | 15 days, 16:00:00 |     4     0     8  33.3 |
| ATOM/USDT |     10 |           2.89 |          28.95 |            57.958 |           5.80 | 19 days, 14:24:00 |     3     0     7  30.0 |
|  UNI/USDT |      6 |           3.64 |          21.81 |            43.668 |           4.37 | 13 days, 12:00:00 |     2     0     4  33.3 |
|  EOS/USDT |      3 |          -0.18 |          -0.53 |            -1.065 |          -0.11 |  10 days, 0:00:00 |     1     0     2  33.3 |
|  XRP/USDT |      3 |          -2.05 |          -6.15 |           -12.322 |          -1.23 | 13 days, 16:00:00 |     1     0     2  33.3 |
|  YFI/USDT |      2 |         -21.44 |         -42.87 |           -85.829 |          -8.58 |   6 days, 0:00:00 |     0     0     2     0 |
| DOGE/USDT |      3 |         -19.07 |         -57.22 |          -114.554 |         -11.46 |   8 days, 0:00:00 |     0     0     3     0 |
|     TOTAL |    113 |           8.99 |        1016.08 |          2034.201 |         203.42 | 16 days, 17:50:00 |    50     0    63  44.2 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |     100 |     50     0    50  50.0 |          13.43 |        1343.03 |           2688.75 |         447.68 |
|     stop_loss |      13 |      0     0    13     0 |         -25.15 |        -326.95 |           -654.55 |        -108.98 |
====================================================== LEFT OPEN TRADES REPORT ======================================================
|   Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|--------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|  TOTAL |      0 |           0.00 |           0.00 |             0.000 |           0.00 |           0:00 |     0     0     0     0 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-05-31 00:00:00 |
| Max open trades       | 3                   |
|                       |                     |
| Total trades          | 113                 |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 3034.201 USDT       |
| Absolute profit       | 2034.201 USDT       |
| Total profit %        | 203.42%             |
| Trades per day        | 0.11                |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 22600.000 USDT      |
|                       |                     |
| Best Pair             | ADA/USDT 220.3%     |
| Worst Pair            | DOGE/USDT -57.22%   |
| Best trade            | ADA/USDT 102.64%    |
| Worst trade           | LINK/USDT -25.15%   |
| Best day              | 205.487 USDT        |
| Worst day             | -72.162 USDT        |
| Days win/draw/lose    | 45 / 855 / 51       |
| Avg. Duration Winners | 20 days, 22:05:00   |
| Avg. Duration Loser   | 13 days, 10:17:00   |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 4720                |
|                       |                     |
| Min balance           | 693.957 USDT        |
| Max balance           | 3169.296 USDT       |
| Drawdown              | 150.44%             |
| Drawdown              | 301.180 USDT        |
| Drawdown high         | -4.862 USDT         |
| Drawdown low          | -306.043 USDT       |
| Drawdown Start        | 2018-10-12 00:00:00 |
| Drawdown End          | 2019-02-05 00:00:00 |
| Market change         | 1627.12%            |
===============================================

```

Dat wordt heel veel tweaken om een beetje leuke profit uit de markt te halen.

# Aanpassingen van de strategie

Nu ga ik wat zaken aan de strategie aanpassen. Als eerste ga ik de SL/TP strategie van 1:1.5 / 1.2 loslaten en long supertrend laten bepalen wanneer stoploss en takeprofit zijn (trend trade).

**1 hour strategy**

```
Result for strategy my_supertrend_strat
========================================================== BACKTESTING REPORT =========================================================
|     Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|----------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
| ETH/USDT |    368 |           0.52 |         191.28 |           382.934 |          38.29 | 1 day, 0:20:00 |   143     0   225  38.9 |
|    TOTAL |    368 |           0.52 |         191.28 |           382.934 |          38.29 | 1 day, 0:20:00 |   143     0   225  38.9 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |     368 |    143     0   225  38.9 |           0.52 |         191.28 |           382.934 |         191.28 |
====================================================== LEFT OPEN TRADES REPORT ======================================================
|   Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|--------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|  TOTAL |      0 |           0.00 |           0.00 |             0.000 |           0.00 |           0:00 |     0     0     0     0 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-06-02 11:00:00 |
| Max open trades       | 1                   |
|                       |                     |
| Total trades          | 368                 |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 1382.934 USDT       |
| Absolute profit       | 382.934 USDT        |
| Total profit %        | 38.29%              |
| Trades per day        | 0.37                |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 73600.000 USDT      |
|                       |                     |
| Best Pair             | ETH/USDT 191.28%    |
| Worst Pair            | ETH/USDT 191.28%    |
| Best trade            | ETH/USDT 27.6%      |
| Worst trade           | ETH/USDT -12.45%    |
| Best day              | 55.259 USDT         |
| Worst day             | -24.926 USDT        |
| Days win/draw/lose    | 141 / 643 / 215     |
| Avg. Duration Winners | 1 day, 12:48:00     |
| Avg. Duration Loser   | 16:24:00            |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 0                   |
|                       |                     |
| Min balance           | 911.220 USDT        |
| Max balance           | 1407.436 USDT       |
| Drawdown              | 46.64%              |
| Drawdown              | 93.365 USDT         |
| Drawdown high         | 4.585 USDT          |
| Drawdown low          | -88.780 USDT        |
| Drawdown Start        | 2018-09-14 09:00:00 |
| Drawdown End          | 2018-12-16 22:00:00 |
| Market change         | 1073.4%             |
===============================================

```

**1 day strategy**

 Lijkt nog de meeste profit op te leveren. Tevens een win rate van 50% op alleen ethereum.
 
 ```
 Result for strategy my_supertrend_strat
=========================================================== BACKTESTING REPORT ===========================================================
|     Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|----------+--------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
| ETH/USDT |     21 |          12.13 |         254.69 |           509.885 |          50.99 | 23 days, 20:34:00 |    10     0    11  47.6 |
|    TOTAL |     21 |          12.13 |         254.69 |           509.885 |          50.99 | 23 days, 20:34:00 |    10     0    11  47.6 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |      20 |     10     0    10  50.0 |          13.99 |         279.84 |           560.235 |         279.84 |
|     stop_loss |       1 |      0     0     1     0 |         -25.15 |         -25.15 |           -50.35  |         -25.15 |
====================================================== LEFT OPEN TRADES REPORT ======================================================
|   Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|--------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|  TOTAL |      0 |           0.00 |           0.00 |             0.000 |           0.00 |           0:00 |     0     0     0     0 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-05-31 00:00:00 |
| Max open trades       | 1                   |
|                       |                     |
| Total trades          | 21                  |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 1509.885 USDT       |
| Absolute profit       | 509.885 USDT        |
| Total profit %        | 50.99%              |
| Trades per day        | 0.02                |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 4200.000 USDT       |
|                       |                     |
| Best Pair             | ETH/USDT 254.69%    |
| Worst Pair            | ETH/USDT 254.69%    |
| Best trade            | ETH/USDT 129.84%    |
| Worst trade           | ETH/USDT -25.15%    |
| Best day              | 259.941 USDT        |
| Worst day             | -50.350 USDT        |
| Days win/draw/lose    | 10 / 929 / 11       |
| Avg. Duration Winners | 33 days, 0:00:00    |
| Avg. Duration Loser   | 15 days, 13:05:00   |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 0                   |
|                       |                     |
| Min balance           | 917.573 USDT        |
| Max balance           | 1509.885 USDT       |
| Drawdown              | 40.77%              |
| Drawdown              | 81.625 USDT         |
| Drawdown high         | 6.621 USDT          |
| Drawdown low          | -75.004 USDT        |
| Drawdown Start        | 2019-06-28 00:00:00 |
| Drawdown End          | 2019-11-22 00:00:00 |
| Market change         | 1157.39%            |
===============================================
 ```

Voor meerdere coins (Binance top 10) is het resultaat op day timeframe:

```
            "BTC/USDT",
            "ETH/USDT",
            "BNB/USDT",
            "ADA/USDT",
            "XRP/USDT",
            "BCH/USDT",
            // "DASH/USDT",
            "UNI/USDT",
            "LTC/USDT",
            "LINK/USDT",
            // "XLM/USDT",
            "DOGE/USDT",
            "AAVE/USDT",
            "VET/USDT",
            "ATOM/USDT",
            "TRX/USDT",
            "YFI/USDT",
            "EOS/USDT",
```

```
Result for strategy my_supertrend_strat
============================================================ BACKTESTING REPORT ===========================================================
|      Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|-----------+--------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|  ADA/USDT |      9 |          38.57 |         347.12 |           694.928 |          69.49 |  30 days, 2:40:00 |     3     0     6  33.3 |
|  VET/USDT |      3 |         101.25 |         303.76 |           608.119 |          60.81 |  36 days, 0:00:00 |     2     0     1  66.7 |
| AAVE/USDT |      2 |         101.43 |         202.87 |           406.140 |          40.61 |  19 days, 0:00:00 |     1     0     1  50.0 |
|  LTC/USDT |      6 |          26.29 |         157.71 |           315.742 |          31.57 |  28 days, 0:00:00 |     4     0     2  66.7 |
|  BNB/USDT |     11 |          14.05 |         154.59 |           309.486 |          30.95 |  27 days, 6:33:00 |     4     0     7  36.4 |
| LINK/USDT |      6 |          21.56 |         129.37 |           258.993 |          25.90 |  28 days, 8:00:00 |     3     0     3  50.0 |
|  ETH/USDT |     14 |           8.14 |         114.01 |           228.257 |          22.83 | 21 days, 17:09:00 |     5     0     9  35.7 |
|  TRX/USDT |      4 |          23.32 |          93.27 |           186.717 |          18.67 | 17 days, 18:00:00 |     3     0     1  75.0 |
|  BTC/USDT |     12 |           7.50 |          90.00 |           180.171 |          18.02 |  24 days, 4:00:00 |     4     0     8  33.3 |
|  BCH/USDT |      2 |          37.45 |          74.91 |           149.967 |          15.00 |  31 days, 0:00:00 |     1     0     1  50.0 |
|  EOS/USDT |      2 |          12.31 |          24.62 |            49.285 |           4.93 |  15 days, 0:00:00 |     1     0     1  50.0 |
|  XRP/USDT |      2 |           8.14 |          16.28 |            32.587 |           3.26 |  18 days, 0:00:00 |     1     0     1  50.0 |
| ATOM/USDT |      6 |           0.79 |           4.73 |             9.474 |           0.95 | 20 days, 20:00:00 |     2     0     4  33.3 |
|  YFI/USDT |      2 |          -0.14 |          -0.29 |            -0.575 |          -0.06 |   9 days, 0:00:00 |     1     0     1  50.0 |
| DOGE/USDT |      3 |         -19.05 |         -57.15 |          -114.420 |         -11.44 |  13 days, 0:00:00 |     0     0     3     0 |
|  UNI/USDT |      3 |         -21.12 |         -63.35 |          -126.826 |         -12.68 | 18 days, 16:00:00 |     0     0     3     0 |
|     TOTAL |     87 |          18.30 |        1592.43 |          3188.046 |         318.80 | 23 days, 23:27:00 |    35     0    52  40.2 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |      77 |     35     0    42  45.5 |          23.95 |        1843.93 |           3691.55 |         614.64 |
|     stop_loss |      10 |      0     0    10     0 |         -25.15 |        -251.5  |           -503.5  |         -83.83 |
====================================================== LEFT OPEN TRADES REPORT ======================================================
|   Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|--------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|  TOTAL |      0 |           0.00 |           0.00 |             0.000 |           0.00 |           0:00 |     0     0     0     0 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-05-31 00:00:00 |
| Max open trades       | 3                   |
|                       |                     |
| Total trades          | 87                  |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 4188.046 USDT       |
| Absolute profit       | 3188.046 USDT       |
| Total profit %        | 318.8%              |
| Trades per day        | 0.09                |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 17400.000 USDT      |
|                       |                     |
| Best Pair             | ADA/USDT 347.12%    |
| Worst Pair            | UNI/USDT -63.35%    |
| Best trade            | VET/USDT 258.91%    |
| Worst trade           | LINK/USDT -25.15%   |
| Best day              | 518.341 USDT        |
| Worst day             | -100.700 USDT       |
| Days win/draw/lose    | 31 / 881 / 39       |
| Avg. Duration Winners | 37 days, 11:39:00   |
| Avg. Duration Loser   | 14 days, 21:14:00   |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 5772                |
|                       |                     |
| Min balance           | 693.957 USDT        |
| Max balance           | 4314.872 USDT       |
| Drawdown              | 150.44%             |
| Drawdown              | 301.180 USDT        |
| Drawdown high         | -4.862 USDT         |
| Drawdown low          | -306.043 USDT       |
| Drawdown Start        | 2018-10-12 00:00:00 |
| Drawdown End          | 2019-02-05 00:00:00 |
| Market change         | 1627.12%            |
===============================================
```

# Tot slot

Helemaal loslaten van de bovengenoemde strategie en alleen supertrend loslaten op coins met day timeframe is kennelijk de meest simpele strategie dat ook nog eens het meeste oplevert...

Winrate is niet spectaculair maar goede trades leveren dan wel gelijk veel op:

```
Result for strategy supertrend_strat_prod
============================================================ BACKTESTING REPORT ===========================================================
|      Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|-----------+--------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|  BNB/USDT |     13 |          83.96 |        1091.52 |          2185.220 |         218.52 |  30 days, 5:32:00 |     5     0     8  38.5 |
|  UNI/USDT |      3 |         212.21 |         636.63 |          1274.528 |         127.45 | 36 days, 16:00:00 |     1     0     2  33.3 |
| LINK/USDT |      9 |          20.45 |         184.02 |           368.399 |          36.84 | 24 days, 16:00:00 |     3     0     6  33.3 |
|  BTC/USDT |     13 |          12.18 |         158.30 |           316.922 |          31.69 | 26 days, 20:18:00 |     4     0     9  30.8 |
|  ADA/USDT |     11 |          13.41 |         147.55 |           295.396 |          29.54 |  28 days, 8:44:00 |     4     0     7  36.4 |
|  LTC/USDT |      3 |          45.66 |         136.98 |           274.244 |          27.42 |  37 days, 8:00:00 |     2     0     1  66.7 |
|  XRP/USDT |      7 |          11.61 |          81.30 |           162.755 |          16.28 |  16 days, 6:51:00 |     2     0     5  28.6 |
| ATOM/USDT |      4 |          17.34 |          69.36 |           138.861 |          13.89 |  34 days, 6:00:00 |     2     0     2  50.0 |
|  EOS/USDT |      2 |          30.50 |          61.00 |           122.122 |          12.21 |  34 days, 0:00:00 |     1     0     1  50.0 |
|  ETH/USDT |     15 |           3.35 |          50.18 |           100.463 |          10.05 | 16 days, 11:12:00 |     6     0     9  40.0 |
|  VET/USDT |      5 |           6.18 |          30.89 |            61.847 |           6.18 |  18 days, 0:00:00 |     1     0     4  20.0 |
|  TRX/USDT |      5 |           3.99 |          19.97 |            39.970 |           4.00 | 22 days, 19:12:00 |     2     0     3  40.0 |
| DOGE/USDT |      5 |           3.72 |          18.59 |            37.210 |           3.72 |  20 days, 9:36:00 |     2     0     3  40.0 |
| AAVE/USDT |      0 |           0.00 |           0.00 |             0.000 |           0.00 |              0:00 |     0     0     0     0 |
|  YFI/USDT |      2 |         -25.15 |         -50.30 |          -100.700 |         -10.07 |  3 days, 12:00:00 |     0     0     2     0 |
|  BCH/USDT |      3 |         -17.87 |         -53.60 |          -107.309 |         -10.73 |   7 days, 0:00:00 |     0     0     3     0 |
|     TOTAL |    100 |          25.82 |        2582.38 |          5169.928 |         516.99 | 23 days, 23:31:00 |    35     0    65  35.0 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |      88 |     34     0    54  38.6 |          21.14 |        1860.03 |           3723.78 |         620.01 |
|     stop_loss |      11 |      0     0    11     0 |         -25.15 |        -276.65 |           -553.85 |         -92.22 |
|           roi |       1 |      1     0     0   100 |         999    |         999    |           2000    |         333    |
====================================================== LEFT OPEN TRADES REPORT ======================================================
|   Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|--------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|  TOTAL |      0 |           0.00 |           0.00 |             0.000 |           0.00 |           0:00 |     0     0     0     0 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-05-31 00:00:00 |
| Max open trades       | 3                   |
|                       |                     |
| Total trades          | 100                 |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 6169.928 USDT       |
| Absolute profit       | 5169.928 USDT       |
| Total profit %        | 516.99%             |
| Trades per day        | 0.1                 |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 20000.000 USDT      |
|                       |                     |
| Best Pair             | BNB/USDT 1091.52%   |
| Worst Pair            | BCH/USDT -53.6%     |
| Best trade            | BNB/USDT 999.0%     |
| Worst trade           | DOGE/USDT -25.15%   |
| Best day              | 2000.000 USDT       |
| Worst day             | -151.050 USDT       |
| Days win/draw/lose    | 26 / 882 / 43       |
| Avg. Duration Winners | 46 days, 6:51:00    |
| Avg. Duration Loser   | 11 days, 23:16:00   |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 7412                |
|                       |                     |
| Min balance           | 728.883 USDT        |
| Max balance           | 6346.743 USDT       |
| Drawdown              | 170.23%             |
| Drawdown              | 340.806 USDT        |
| Drawdown high         | 631.806 USDT        |
| Drawdown low          | 291.001 USDT        |
| Drawdown Start        | 2019-06-28 00:00:00 |
| Drawdown End          | 2019-11-16 00:00:00 |
| Market change         | 1627.12%            |

```

```
# --- Do not remove these libs ---
from freqtrade.strategy.interface import IStrategy
from pandas import DataFrame
# --------------------------------
import talib.abstract as ta
import pandas_ta as pta
import freqtrade.vendor.qtpylib.indicators as qtpylib
# ---------- Commands -----------
# Commands for backtesting etc.:
# /opt/freqtrade/.env/bin/freqtrade backtesting --config user_data/configs/backtest_conf.json --strategy new_strat
# /opt/freqtrade/.env/bin/freqtrade backtesting --config user_data/configs/backtest_conf.json --timerange=20210201-20210310 --strategy new_strat --export trades --export-filename=user_data/backtest_results/simple_strat_30m_test.json
# /opt/freqtrade/.env/bin/freqtrade plot-dataframe --config user_data/configs/backtest_conf.json --strategy new_strat  --export-filename=user_data/backtest_results/new_strat-2021-05-13_19-29-25.json
# /opt/freqtrade/.env/bin/freqtrade plot-profit --config user_data/configs/backtest_conf.json  --strategy new_strat --export-filename=user_data/backtest_results/new_strat-2021-05-13_19-29-25.json
# --------------------------------

class supertrend_strat_prod(IStrategy):
    stoploss = -0.25
    timeframe = "1d"

    order_types = {
        "buy": "limit",
        "sell": "limit",
        "emergencysell": "market",
        "stoploss": "market",
        "stoploss_on_exchange": False,
        "stoploss_on_exchange_interval": 60,
        "stoploss_on_exchange_limit_ratio": 0.99,
    }

    def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        # Indicatoren
        dataframe['supertrend'] = pta.supertrend(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'], length=7, multiplier=2.0)['SUPERT_7_2.0']

        return dataframe

    def populate_buy_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
            (dataframe['close'] > dataframe['supertrend'])
            ),
            "buy",
        ] = 1

        return dataframe

    def populate_sell_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
            (dataframe['close'] < dataframe['supertrend'])            ),
            "sell",
        ] = 1
        return dataframe



```

Alleen al supertrend met 10/3.5 gebruiken op hourly timeframe levert al behoorlijk veel op:

```
Result for strategy test
=========================================================== BACKTESTING REPORT ===========================================================
|      Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |
|-----------+--------+----------------+----------------+-------------------+----------------+------------------+-------------------------|
|  BNB/USDT |    108 |           3.22 |         347.88 |           696.458 |          69.65 | 2 days, 15:14:00 |    48     0    60  44.4 |
| DOGE/USDT |     37 |           9.01 |         333.55 |           667.765 |          66.78 |  2 days, 5:36:00 |     9     0    28  24.3 |
|  ADA/USDT |     91 |           3.51 |         319.36 |           639.353 |          63.94 | 2 days, 17:20:00 |    34     0    57  37.4 |
| LINK/USDT |     75 |           2.82 |         211.39 |           423.210 |          42.32 | 2 days, 22:42:00 |    30     0    45  40.0 |
| ATOM/USDT |     54 |           3.47 |         187.30 |           374.972 |          37.50 | 2 days, 13:56:00 |    23     0    31  42.6 |
| AAVE/USDT |     14 |          11.61 |         162.51 |           325.336 |          32.53 | 3 days, 13:21:00 |     8     0     6  57.1 |
|  XRP/USDT |     67 |           1.89 |         126.39 |           253.035 |          25.30 |  2 days, 7:30:00 |    17     0    50  25.4 |
|  BTC/USDT |    119 |           1.05 |         125.44 |           251.122 |          25.11 | 2 days, 11:36:00 |    46     0    73  38.7 |
|  UNI/USDT |     18 |           6.91 |         124.39 |           249.020 |          24.90 |  3 days, 1:03:00 |     7     0    11  38.9 |
|  ETH/USDT |    111 |           1.01 |         111.67 |           223.569 |          22.36 |  2 days, 4:25:00 |    29     0    82  26.1 |
|  YFI/USDT |     18 |           4.61 |          82.98 |           166.121 |          16.61 |  2 days, 7:43:00 |     6     0    12  33.3 |
|  LTC/USDT |     64 |           0.79 |          50.25 |           100.597 |          10.06 | 2 days, 13:52:00 |    25     0    39  39.1 |
|  TRX/USDT |     45 |           0.79 |          35.57 |            71.203 |           7.12 | 2 days, 13:36:00 |    20     0    25  44.4 |
|  BCH/USDT |     36 |           0.97 |          34.82 |            69.717 |           6.97 |  2 days, 9:05:00 |    14     0    22  38.9 |
|  VET/USDT |     70 |           0.13 |           9.21 |            18.430 |           1.84 |  2 days, 0:09:00 |    22     0    48  31.4 |
|  EOS/USDT |     38 |          -0.08 |          -3.17 |            -6.340 |          -0.63 |  2 days, 0:05:00 |     9     0    29  23.7 |
|     TOTAL |    965 |           2.34 |        2259.52 |          4523.568 |         452.36 | 2 days, 11:37:00 |   347     0   618  36.0 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |     960 |    344     0   616  35.8 |           2.36 |        2268.75 |          4542.03  |         756.25 |
|    force_sell |       3 |      3     0     0   100 |          13.69 |          41.08 |            82.233 |          13.69 |
|     stop_loss |       2 |      0     0     2     0 |         -25.15 |         -50.3  |          -100.7   |         -16.77 |
======================================================= LEFT OPEN TRADES REPORT ========================================================
|     Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |    Avg Duration |   Win  Draw  Loss  Win% |
|----------+--------+----------------+----------------+-------------------+----------------+-----------------+-------------------------|
| ADA/USDT |      1 |          15.63 |          15.63 |            31.286 |           3.13 | 3 days, 6:00:00 |     1     0     0   100 |
| BNB/USDT |      1 |          15.05 |          15.05 |            30.132 |           3.01 | 3 days, 3:00:00 |     1     0     0   100 |
| VET/USDT |      1 |          10.40 |          10.40 |            20.816 |           2.08 | 3 days, 4:00:00 |     1     0     0   100 |
|    TOTAL |      3 |          13.69 |          41.08 |            82.233 |           8.22 | 3 days, 4:20:00 |     3     0     0   100 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-06-02 11:00:00 |
| Max open trades       | 3                   |
|                       |                     |
| Total trades          | 965                 |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 5523.568 USDT       |
| Absolute profit       | 4523.568 USDT       |
| Total profit %        | 452.36%             |
| Trades per day        | 0.97                |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 193000.000 USDT     |
|                       |                     |
| Best Pair             | BNB/USDT 347.88%    |
| Worst Pair            | EOS/USDT -3.17%     |
| Best trade            | DOGE/USDT 402.71%   |
| Worst trade           | YFI/USDT -25.15%    |
| Best day              | 787.953 USDT        |
| Worst day             | -124.438 USDT       |
| Days win/draw/lose    | 188 / 574 / 237     |
| Avg. Duration Winners | 4 days, 14:57:00    |
| Avg. Duration Loser   | 1 day, 6:48:00      |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 170255              |
|                       |                     |
| Min balance           | 765.357 USDT        |
| Max balance           | 5523.568 USDT       |
| Drawdown              | 174.43%             |
| Drawdown              | 349.201 USDT        |
| Drawdown high         | 114.558 USDT        |
| Drawdown low          | -234.643 USDT       |
| Drawdown Start        | 2018-09-30 21:00:00 |
| Drawdown End          | 2018-12-14 16:00:00 |
| Market change         | 1741.82%            |
===============================================
```

En wat dacht je van 1Day timeframe:

```
Result for strategy test
============================================================ BACKTESTING REPORT ===========================================================
|      Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|-----------+--------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|  BNB/USDT |      6 |         200.75 |        1204.51 |          2411.424 |         241.14 |  69 days, 4:00:00 |     3     0     3  50.0 |
|  ADA/USDT |      6 |         196.95 |        1181.70 |          2365.754 |         236.58 | 67 days, 12:00:00 |     3     0     3  50.0 |
| LINK/USDT |      4 |         111.87 |         447.49 |           895.883 |          89.59 |  72 days, 6:00:00 |     3     0     1  75.0 |
|  BTC/USDT |      6 |          51.15 |         306.92 |           614.449 |          61.44 | 58 days, 16:00:00 |     2     0     4  33.3 |
|  ETH/USDT |      9 |           6.53 |          58.73 |           117.575 |          11.76 | 40 days, 18:40:00 |     3     0     6  33.3 |
|  BCH/USDT |      3 |           8.55 |          25.66 |            51.366 |           5.14 |  29 days, 8:00:00 |     1     0     2  33.3 |
| ATOM/USDT |      3 |           5.75 |          17.24 |            34.519 |           3.45 |  49 days, 8:00:00 |     1     0     2  33.3 |
|  TRX/USDT |      2 |           7.14 |          14.27 |            28.577 |           2.86 | 70 days, 12:00:00 |     1     0     1  50.0 |
|  LTC/USDT |      0 |           0.00 |           0.00 |             0.000 |           0.00 |              0:00 |     0     0     0     0 |
|  VET/USDT |      1 |          -2.83 |          -2.83 |            -5.656 |          -0.57 |  24 days, 0:00:00 |     0     0     1     0 |
|  EOS/USDT |      1 |         -25.15 |         -25.15 |           -50.350 |          -5.04 |   9 days, 0:00:00 |     0     0     1     0 |
|  XRP/USDT |      4 |          -7.43 |         -29.72 |           -59.503 |          -5.95 | 24 days, 12:00:00 |     1     0     3  25.0 |
|  YFI/USDT |      2 |         -19.35 |         -38.71 |           -77.497 |          -7.75 | 14 days, 12:00:00 |     0     0     2     0 |
| AAVE/USDT |      2 |         -25.15 |         -50.30 |          -100.700 |         -10.07 |          12:00:00 |     0     0     2     0 |
| DOGE/USDT |      4 |         -15.33 |         -61.33 |          -122.786 |         -12.28 |  23 days, 0:00:00 |     0     0     4     0 |
|  UNI/USDT |      3 |         -25.15 |         -75.45 |          -151.050 |         -15.11 |  12 days, 0:00:00 |     0     0     3     0 |
|     TOTAL |     56 |          53.09 |        2973.03 |          5952.006 |         595.20 | 44 days, 12:51:00 |    18     0    38  32.1 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+--------------------------+----------------+----------------+-------------------+----------------|
|   sell_signal |      38 |     16     0    22  42.1 |          36.25 |        1377.43 |           2757.61 |         459.14 |
|     stop_loss |      16 |      0     0    16     0 |         -25.15 |        -402.4  |           -805.6  |        -134.13 |
|           roi |       2 |      2     0     0   100 |         999    |        1998    |           4000    |         666    |
====================================================== LEFT OPEN TRADES REPORT ======================================================
|   Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|--------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|  TOTAL |      0 |           0.00 |           0.00 |             0.000 |           0.00 |           0:00 |     0     0     0     0 |
=============== SUMMARY METRICS ===============
| Metric                | Value               |
|-----------------------+---------------------|
| Backtesting from      | 2018-09-07 00:00:00 |
| Backtesting to        | 2021-05-31 00:00:00 |
| Max open trades       | 3                   |
|                       |                     |
| Total trades          | 56                  |
| Starting balance      | 1000.000 USDT       |
| Final balance         | 6952.006 USDT       |
| Absolute profit       | 5952.006 USDT       |
| Total profit %        | 595.2%              |
| Trades per day        | 0.06                |
| Avg. stake amount     | 200.000 USDT        |
| Total trade volume    | 11200.000 USDT      |
|                       |                     |
| Best Pair             | BNB/USDT 1204.51%   |
| Worst Pair            | UNI/USDT -75.45%    |
| Best trade            | BNB/USDT 999.0%     |
| Worst trade           | ETH/USDT -25.15%    |
| Best day              | 2000.000 USDT       |
| Worst day             | -151.050 USDT       |
| Days win/draw/lose    | 17 / 886 / 25       |
| Avg. Duration Winners | 96 days, 4:00:00    |
| Avg. Duration Loser   | 20 days, 1:54:00    |
| Zero Duration Trades  | 0.00% (0)           |
| Rejected Buy signals  | 7321                |
|                       |                     |
| Min balance           | 846.189 USDT        |
| Max balance           | 7174.306 USDT       |
| Drawdown              | 149.17%             |
| Drawdown              | 298.645 USDT        |
| Drawdown high         | 2056.179 USDT       |
| Drawdown low          | 1757.533 USDT       |
| Drawdown Start        | 2020-09-06 00:00:00 |
| Drawdown End          | 2020-11-04 00:00:00 |
| Market change         | 1627.12%            |
===============================================

```

Config:
```
# --- Do not remove these libs ---
from freqtrade.strategy.interface import IStrategy
from pandas import DataFrame
# --------------------------------
import talib.abstract as ta
import pandas_ta as pta
import freqtrade.vendor.qtpylib.indicators as qtpylib
# ---------- Commands -----------
# Commands for backtesting etc.:
# /opt/freqtrade/.env/bin/freqtrade backtesting --config user_data/configs/backtest_conf.json --strategy new_strat
# /opt/freqtrade/.env/bin/freqtrade backtesting --config user_data/configs/backtest_conf.json --timerange=20210201-20210310 --strategy new_strat --export trades --export-filename=user_data/backtest_results/simple_strat_30m_test.json
# /opt/freqtrade/.env/bin/freqtrade plot-dataframe --config user_data/configs/backtest_conf.json --strategy new_strat  --export-filename=user_data/backtest_results/new_strat-2021-05-13_19-29-25.json
# /opt/freqtrade/.env/bin/freqtrade plot-profit --config user_data/configs/backtest_conf.json  --strategy new_strat --export-filename=user_data/backtest_results/new_strat-2021-05-13_19-29-25.json
# --------------------------------

class test(IStrategy):
    stoploss = -0.25
    timeframe = "1h"

    order_types = {
        "buy": "limit",
        "sell": "limit",
        "emergencysell": "market",
        "stoploss": "market",
        "stoploss_on_exchange": False,
        "stoploss_on_exchange_interval": 60,
        "stoploss_on_exchange_limit_ratio": 0.99,
    }

    def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        # Indicatoren
        # dataframe['supertrend'] = pta.supertrend(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'], length=7, multiplier=2.0)['SUPERT_7_2.0']
        dataframe['supertrend'] = pta.supertrend(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'], length=10, multiplier=3.5)['SUPERT_10_3.5']

        return dataframe

    def populate_buy_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
            (dataframe['close'] > dataframe['supertrend'])
            ),
            "buy",
        ] = 1

        return dataframe

    def populate_sell_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (
            (dataframe['close'] < dataframe['supertrend'])            ),
            "sell",
        ] = 1
        return dataframe



```

De grote vraag is waar 'slechte' trades zich bevinden en welke indicator die kan tegengaan?

SSL misschien?

Dit ga ik in een ander notebook als verdere uitdaging oppakken.

# The end